In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,random_split
import torch.optim as optim
from torchvision.transforms import transforms
from PIL import Image
import scipy
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from torch.utils.data import Subset

from utils import OxfordDataset

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
transform_v = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
])

transform_t = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(p=0.5),        # Flip 50% of images
    transforms.RandomRotation(degrees=15),         # Rotate ±15 degrees
    transforms.ColorJitter(                        # Color augmentation
        brightness=0.2,
        contrast=0.2,
        saturation=0.2,
        hue=0.1
    ),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
])

In [4]:
train_dataset = OxfordDataset("flower_data", transform=transform_t)
validation_dataset = OxfordDataset("flower_data", transform=transform_v)
test_dataset = OxfordDataset("flower_data", transform=transform_v)


In [5]:
train_split = int(0.70 * len(train_dataset))
validation_split = int(0.15 * len(train_dataset))
test_split = len(train_dataset) - train_split - validation_split





In [6]:
indices = torch.randperm(len(train_dataset),generator=torch.Generator().manual_seed(42)).tolist()

train_indices = indices[:train_split]
validation_indices = indices[train_split:train_split+validation_split]
test_indices = indices[train_split+validation_split:]


In [7]:
train_dataset = Subset(train_dataset,train_indices)
val_dataset = Subset(validation_dataset,validation_indices)
test_dataset = Subset(test_dataset,test_indices)

In [8]:
test_indices.__len__()

1229

In [9]:
len(indices)

8189

In [10]:
train_loader = DataLoader(train_dataset,batch_size=32, shuffle=True)
validation_loader = DataLoader(val_dataset,batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [11]:
for image,label in train_loader:
    print(label)
    print(label[0].item())
    print(image.shape)
    break



tensor([88, 50, 71, 82, 57, 41, 12, 54, 27, 47, 93, 57, 70, 83, 39, 70, 65, 77,
        78, 76, 76, 54, 16, 76, 79, 96, 42, 45, 43, 72, 94, 66],
       dtype=torch.uint8)
88
torch.Size([32, 3, 224, 224])


In [12]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.convBlock = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64,kernel_size=3,padding=1,stride=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2) ,# 224 -> 112

            nn.Conv2d(in_channels=64, out_channels=128,kernel_size=3,stride=2,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2) ,# 112 -> 56 -> 28

            nn.Conv2d(in_channels=128, out_channels=128,kernel_size=3,stride=2,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # 28 -> 14 -> 7            
        )

        self.fcBlock = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128*3*3,1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024,102),
        )

    def forward(self, x):
        x = self.convBlock(x)
        return self.fcBlock(x)



In [13]:
model = SimpleCNN().to(device)

optimizer = optim.Adam(model.parameters(),lr=0.0005, weight_decay=0.0005)
loss_function = nn.CrossEntropyLoss()


In [14]:
def train_epoch(model, train_loader, optimizer, loss_function):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for batch_idx, (data, labels) in enumerate(train_loader):
        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_function(output, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = output.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        if (batch_idx + 1) % 10 == 0:  # Changed condition
            avg_loss = running_loss / 10  # Now correct
            acc = 100. * correct / total
            print(f' [{(batch_idx + 1) * len(data)}/{len(train_loader.dataset)}]'
                  f' Loss: {avg_loss:.3f} | Accuracy: {acc:.1f} %')
            running_loss = 0.0  # Only reset loss, keep accuracy cumulative

In [15]:
def validation(model,validation_loader,device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for idx, (data, targets) in enumerate(validation_loader):
            data, targets = data.to(device), targets.to(device)
            output = model(data)
            _,predicted = output.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    
    return 100. * correct/total


In [19]:
num_epochs = 30

for idx in range(num_epochs):
    print(f"Epoch : {idx}")
    train_epoch(model,train_loader,optimizer,loss_function)
    validation_acc = validation(model, validation_loader, device)
    print(f"Validation Accuracy : {validation_acc}")



Epoch : 0
 [320/5732] Loss: 0.609 | Accuracy: 83.1 %
 [640/5732] Loss: 0.628 | Accuracy: 81.2 %
 [960/5732] Loss: 0.655 | Accuracy: 80.5 %
 [1280/5732] Loss: 0.651 | Accuracy: 81.1 %
 [1600/5732] Loss: 0.614 | Accuracy: 81.0 %
 [1920/5732] Loss: 0.705 | Accuracy: 80.8 %
 [2240/5732] Loss: 0.714 | Accuracy: 80.1 %
 [2560/5732] Loss: 0.613 | Accuracy: 80.3 %
 [2880/5732] Loss: 0.592 | Accuracy: 80.2 %
 [3200/5732] Loss: 0.776 | Accuracy: 79.7 %
 [3520/5732] Loss: 0.750 | Accuracy: 79.6 %
 [3840/5732] Loss: 0.635 | Accuracy: 79.7 %
 [4160/5732] Loss: 0.754 | Accuracy: 79.6 %
 [4480/5732] Loss: 0.784 | Accuracy: 79.4 %
 [4800/5732] Loss: 0.659 | Accuracy: 79.4 %
 [5120/5732] Loss: 0.737 | Accuracy: 79.3 %
 [5440/5732] Loss: 0.784 | Accuracy: 79.3 %
 [720/5732] Loss: 0.816 | Accuracy: 79.3 %
Validation Accuracy : 64.98371335504886
Epoch : 1
 [320/5732] Loss: 0.699 | Accuracy: 78.8 %
 [640/5732] Loss: 0.685 | Accuracy: 78.4 %
 [960/5732] Loss: 0.599 | Accuracy: 79.6 %
 [1280/5732] Loss: 0.64

In [20]:
test_acc = validation(model,test_loader,device)
print(test_acc)

68.10414971521563
